In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
from splinter import Browser

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client.drop_database('mars_db')

In [3]:
db = client.mars_db
collection = db.articles

In [4]:
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
There is no [win32] chromedriver for browser 91.0.4472 in cache
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.19/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\katie\.wdm\drivers\chromedriver\win32\91.0.4472.19]


In [5]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# NASA Mars News

mars = {}

results = soup.find_all('div', class_='content_title')
title = results[1].a.text

results = soup.find_all('div', class_= 'article_teaser_body')
para = results[0].text

mars['news_title'] = title
mars['news_paragraph'] = para


In [7]:
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

In [8]:
# JPL Mars Space Image

browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = browser.find_by_tag('button')[1]
results.click()
img_results = soup.find_all('img')[1]['src']
mars['featured_image_url'] = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + img_results


In [9]:
url = 'https://space-facts.com/mars/'

In [10]:
# Mars Facts

table = pd.read_html(url)[0]
table.columns=['Description', 'Mars']
table.set_index('Description', inplace = True)
mars_facts = table.to_html('templates/Mars_fact_table.html')

In [11]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [12]:
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_ = 'description')

img_urls = []

for name in results:
    temp = name.find('h3').text
    two_words = temp[:-9]

    browser.click_link_by_partial_text(temp)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find_all('img')[5]['src']
    base_url = 'astrogeology.usgs.gov'
    img_url = base_url + img
    dic_pic = {'title':two_words,'img_url':img_url}
    
    img_urls.append(dic_pic)
    
    browser.back()
    
for i in len(img_url):
    mars['title'].append(img_url[i].title)
    mars['img_url'].append(img_url[i].img_url)

    

C:\Users\katie\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [13]:
browser.quit()

In [14]:
collection.insert_one(mars)